<a href="https://colab.research.google.com/github/Aniket7773/Signature-Verification-Model/blob/main/sign_verify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# Define the root directory path
root_dir = "/content/drive/MyDrive/Colab Notebooks/sign_data/train"

# Initialize empty lists to store file paths and labels
file_paths = []
labels = []
image=[]
# Function to label images and append to lists
def label_images(directory, label):
    for filename in os.listdir(directory):
        if filename.endswith((".png",".PNG")):  # Assuming your images are in PNG format
            image_path = os.path.join(directory, filename)
            #print(image_path)
            file_paths.append(image_path)
            labels.append(label)

# Traverse the directory structure and label images
for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)

    if os.path.isdir(subdir_path):
        if subdir.endswith("_forg"):
            label = 0  # Forgery label
            #print(label)
        else:
            label = 1  # Genuine label

        image_dir = os.path.join(subdir_path)
        #image.append(image_dir)
        if os.path.exists(image_dir):
            image.append(image_dir)
            label_images(image_dir, label)

print(labels)
print(len(file_paths))
#print(len(image))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load and preprocess the images
def load_and_preprocess_image(file_path, label):
    img = cv2.imread(file_path)
    img = cv2.resize(img, (224, 224))  # Resize to VGG16 input size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = img / 255.0  # Normalize pixel values to [0, 1]
    return img, label

# Create a list of image data and labels
data = []
for file_path, label in zip(file_paths, labels):
    img, label = load_and_preprocess_image(file_path, label)
    data.append((img, label))

# Split data into training and validation sets
X, y = zip(*data)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Load the VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new model and add the VGG16 base model
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # 1 output node for binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Convert data to NumPy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
y_train = np.array(y_train)
y_val = np.array(y_val)

# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=2,  # Adjust the number of epochs as needed
    batch_size=32,  # Adjust batch size as needed
    verbose=1
)
model.save('signature verification.h5')
# # Evaluate the model
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print(f'Test Accuracy: {test_accuracy}')


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/2
42/42 [==============================] - 3153s 75s/step - loss: 0.5857 - accuracy: 0.6641 - val_loss: 0.2367 - val_accuracy: 0.9121
Epoch 2/2
42/42 [==============================] - 3048s 73s/step - loss: 0.2234 - accuracy: 0.9196 - val_loss: 0.0715 - val_accuracy: 0.9758


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('signature verification.h5')


In [ ]:
# Define the root directory path for test data
test_dir = "/content/drive/MyDrive/Colab Notebooks/sign_data/test"


In [ ]:
test_labels = []
test_file_paths = []

# Function to label test images and append to lists
def label_test_images(directory, label):
    for filename in os.listdir(directory):
        if filename.endswith((".png", ".PNG")):  # Check for both .png and .jpg extensions
            image_path = os.path.join(directory, filename)
            test_file_paths.append(image_path)
            test_labels.append(label)

# Traverse the directory structure and label test images
for subdir in os.listdir(test_dir):
    subdir_path = os.path.join(test_dir, subdir)

    if os.path.isdir(subdir_path):
        if subdir.endswith("_forg"):
            label = 0  # Forgery label
        else:
            label = 1  # Genuine label

        label_test_images(subdir_path, label)
print(len(test_labels))

500


In [ ]:
# Create a list of image data and labels for the test dataset
test_data = []
for file_path, label in zip(test_file_paths, test_labels):
    img, label = load_and_preprocess_image(file_path, label)
    test_data.append((img, label))

# Convert test data to NumPy arrays
X_test, y_test = zip(*test_data)
X_test = np.array(X_test)
y_test = np.array(y_test)


In [ ]:
# Evaluate the model using the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')


16/16 [==============================] - 279s 17s/step - loss: 0.0425 - accuracy: 0.9840
Test Accuracy: 0.984000027179718
